In [ ]:
import os
import numpy as np
import cv2
from glob import glob
def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=10):
    name = video_path.split("/")[-1].split(".")[0]

  
  
    print(name)
    if (name.find("Bhuj") != -1):
          save_path = os.path.join(save_dir, "Bhujangasana")
    elif (name.find("Padam") != -1):
          save_path = os.path.join(save_dir, "Padamasana")
    elif (name.find("Shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana")
    elif (name.find("Trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana")
    elif (name.find("Vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana")
    else:
        save_path = os.path.join(save_dir, "Tadasana")
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if idx == 0:
            cv2.imwrite(f"{save_path}/{idx}.png", frame)
        else:
            if idx % gap == 0:
                cv2.imwrite(f"{save_path}/{idx}.png", frame)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("C:/Users/radha/OneDrive/Desktop/BE/Project/Yoga_Vid_Collected/*")
    save_dir = "save"

    for path in video_paths:
        save_frame(path, save_dir, gap=10)

Yoga_Vid_Collected\Abhay_Bhuj
save\Bhujangasana
Yoga_Vid_Collected\Abhay_Padam
save\Padamasana
Yoga_Vid_Collected\Abhay_Shav
save\Shavasana
Yoga_Vid_Collected\Abhay_Tada
save\Tadasana
Yoga_Vid_Collected\Abhay_Trik
save\Trikonasana
Yoga_Vid_Collected\Abhay_Vriksh
save\Vrikshasana
Yoga_Vid_Collected\Ameya_Bhuj
save\Bhujangasana
Yoga_Vid_Collected\Ameya_Padam
save\Padamasana
Yoga_Vid_Collected\Ameya_Shav
save\Shavasana
Yoga_Vid_Collected\Ameya_Tadasan
save\Tadasana
Yoga_Vid_Collected\Ameya_Trikonasana
save\Trikonasana
Yoga_Vid_Collected\Ameya_Vriksh
save\Vrikshasana
Yoga_Vid_Collected\Bhumi_Bhuj
save\Bhujangasana
Yoga_Vid_Collected\Bhumi_Padam
save\Padamasana
Yoga_Vid_Collected\Bhumi_Shav
save\Shavasana
Yoga_Vid_Collected\Bhumi_Tad
save\Tadasana
Yoga_Vid_Collected\Bhumi_Trik
save\Trikonasana
Yoga_Vid_Collected\Bhumi_Vriksh
save\Vrikshasana
Yoga_Vid_Collected\deepa_bhujan
save\Tadasana
Yoga_Vid_Collected\deepa_padmasan
save\Tadasana
Yoga_Vid_Collected\deepa_shava
save\Tadasana
Yoga_Vid_Col